In [11]:
from src.sec_8k_preprocess import load_8k_filings, parse_8k_filing

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
df_8k = load_8k_filings()
df_8k.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
⬇️  Downloading ZIP from SEC...
✅ Download complete.


Parsing SEC JSONs: 100%|██████████| 942738/942738 [01:40<00:00, 9388.05it/s] 


💾 Saving 1,763,864 rows to data/preprocessed/submissions_8k.parquet...
✅ Done.


accession,cik_int,company_name,form,acceptance_datetime,accession_no_dash,url_txt
str,str,str,str,str,str,str
"""0000950131-95-002984""","""5405""","""AMERICAN MAIZE PRODUCTS CO""","""8-K""","""1995-10-27T00:00:00.000Z""","""000095013195002984""","""https://www.sec.gov/Archives/e…"
"""0000950109-95-002831""","""5405""","""AMERICAN MAIZE PRODUCTS CO""","""8-K""","""1995-07-28T00:00:00.000Z""","""000095010995002831""","""https://www.sec.gov/Archives/e…"
"""0000890613-95-000092""","""5405""","""AMERICAN MAIZE PRODUCTS CO""","""8-K""","""1995-07-11T00:00:00.000Z""","""000089061395000092""","""https://www.sec.gov/Archives/e…"
"""0000950123-95-001123""","""5405""","""AMERICAN MAIZE PRODUCTS CO""","""8-K""","""1995-04-25T00:00:00.000Z""","""000095012395001123""","""https://www.sec.gov/Archives/e…"
"""0000950123-95-001008""","""5405""","""AMERICAN MAIZE PRODUCTS CO""","""8-K""","""1995-04-13T00:00:00.000Z""","""000095012395001008""","""https://www.sec.gov/Archives/e…"


In [12]:
# Choose random link for testing
link = df_8k.sample(1)["url_txt"][0]
parse_8k_filing(link)

Downloaded filing from https://www.sec.gov/Archives/edgar/data/700764/000162828015000519/0001628280-15-000519.txt
Parsed filing: https://www.sec.gov/Archives/edgar/data/700764/000162828015000519/0001628280-15-000519.txt with 2 items.


,item,itemText,cik,conm,edgar.link
0,Item 8.01,"Item 8.01 Other EventsOn February 4, 2015, Vic...",0000700764,VICTORY ENERGY CORP CENTRAL INDEX KEY: 000...,https://www.sec.gov/Archives/edgar/data/700764...
1,Item 9.01,Item 9.01 Financial Statements and Exhibits. ...,0000700764,VICTORY ENERGY CORP CENTRAL INDEX KEY: 000...,https://www.sec.gov/Archives/edgar/data/700764...
